In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P0', 'P1']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    #clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P2.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    !rm autoclass.r-params
    !cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/H.csv', index = True, header=True)

2020-04-27 14:59:16 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P0.tsv' as 'real location' with error 0.01
2020-04-27 14:59:16 INFO     Detected encoding: ascii
2020-04-27 14:59:16 INFO     Found 3200 rows and 2 columns
2020-04-27 14:59:16 DEBUG    Checking column names
2020-04-27 14:59:16 DEBUG    Index name 'index'
2020-04-27 14:59:16 DEBUG    Column name 'P0'
2020-04-27 14:59:16 INFO     Checking data format
2020-04-27 14:59:16 INFO     Column 'P0'
2020-04-27 14:59:16 INFO     count    3.200000e+03
2020-04-27 14:59:16 INFO     mean    -7.765439e+03
2020-04-27 14:59:16 INFO     std      1.036135e+06
2020-04-27 14:59:16 INFO     min     -1.783918e+06
2020-04-27 14:59:16 INFO     50%      6.245000e+03
2020-04-27 14:59:16 INFO     max      1.786032e+06
2020-04-27 14:59:16 INFO     ---
2020-04-27 14:59:16 INFO     Final dataframe has 3200 lines and 2 columns
2020-04-27 14:59:16 INFO     Searching for missing values
2020-04-27 14:59:16

Time: 206 sec.

2020-04-27 15:02:43 INFO     Extracting autoclass results
2020-04-27 15:02:43 INFO     Found 3200 cases classified in 12 classes
2020-04-27 15:02:45 INFO     Aggregating input data
2020-04-27 15:02:45 INFO     Writing classes + probabilities .tsv file
2020-04-27 15:02:45 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P1.tsv' as 'real location' with error 0.01
2020-04-27 15:02:45 INFO     Detected encoding: ascii
2020-04-27 15:02:45 INFO     Found 3200 rows and 2 columns
2020-04-27 15:02:45 DEBUG    Checking column names
2020-04-27 15:02:45 DEBUG    Index name 'index'
2020-04-27 15:02:45 DEBUG    Column name 'P1'
2020-04-27 15:02:45 INFO     Checking data format
2020-04-27 15:02:45 INFO     Column 'P1'
2020-04-27 15:02:45 INFO     count    3200.000000
2020-04-27 15:02:45 INFO     mean       -0.300688
2020-04-27 15:02:45 INFO     std         3.208397
2020-04-27 15:02:45 INFO     min        -3.908736
2020-04-27 15:02:45 INFO     50%      

[0.06798245614035088, 0.06263048016701461, 0.056338028169014086, 0.07951070336391437, 0.12987012987012986, 0.11524163568773234, 0.15985130111524162, 0.08064516129032258, 0.10285714285714286, 0.3559322033898305, 0.7857142857142857, 0.42857142857142855]
['EBAY', 'AMAZON', 'DNS', 'MICROSOFT', 'APPLE_ITUNES', 'OFFICE_365', 'APPLE', 'WIKIPEDIA', 'INSTAGRAM', 'INSTAGRAM', 'APPLE_ITUNES', 'NETFLIX']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
0.202095413028034


2020-04-27 15:02:46 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:02:46 INFO     Writing run file
2020-04-27 15:02:46 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:02:46 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 15:02:46 INFO     Running clustering...


Time: 136 sec.

2020-04-27 15:05:02 INFO     Extracting autoclass results
2020-04-27 15:05:02 INFO     Found 3200 cases classified in 7 classes
2020-04-27 15:05:05 INFO     Aggregating input data
2020-04-27 15:05:05 INFO     Writing classes + probabilities .tsv file


[0.047651463580667124, 0.12, 0.08349900596421471, 0.0893854748603352, 0.45714285714285713, 0.06802721088435375, 0.11797752808988764]
['CONTENT_FLASH', 'DNS', 'WHATSAPP', 'IP_ICMP', 'DROPBOX', 'AMAZON', 'APPLE_ICLOUD']
[1, 2, 3, 4, 5, 6, 7]
0.14052622007461651
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2 non-null      object 
 1   1       2 non-null      float64
dtypes: float64(1), object(1)
memory usage: 160.0+ bytes
